In [1]:
import pandas as pd
import os
import numpy as np
pd.set_option('display.max_columns', 500)
# from data_preparation import prepare_triplogs, prepare_tripdatas, get_clean_triplogs, get_clean_tripdatas

In [2]:
!ls ../../../trips/2020_11_8_to_2020_11_14/CVP/triplogs.csv
!ls ../../../trips/2020_11_8_to_2020_11_14/CVP/tripdatas.csv
!ls ../../../trips/2020_11_8_to_2020_11_14/CMS/opticomdevicelog.csv

../../../trips/2020_11_8_to_2020_11_14/CVP/triplogs.csv
../../../trips/2020_11_8_to_2020_11_14/CVP/tripdatas.csv
../../../trips/2020_11_8_to_2020_11_14/CMS/opticomdevicelog.csv


In [12]:
#  load data
triplogs = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CVP/triplogs.csv')
tripdatas = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CVP/tripdatas.csv')
devicelog = pd.read_csv('../../../trips/2020_11_8_to_2020_11_14/CMS/opticomdevicelog.csv')

In [13]:
# create unique trip ID
triplogs['uid'] = triplogs['deviceid'] + "_" + triplogs['logid']
tripdatas['uid'] = tripdatas['deviceid'] + "_" + tripdatas['logid']

# convert to datetime and set time zone
triplogs['starttime'] = pd.to_datetime(triplogs['starttime']).dt.tz_localize('UTC')
triplogs['endtime'] = pd.to_datetime(triplogs['endtime']).dt.tz_localize('UTC')
tripdatas['time'] = pd.to_datetime(tripdatas['time']).dt.tz_localize('UTC')

devicelog['startdatetime'] = pd.to_datetime(devicelog['startdatetime']).dt.tz_localize('US/Pacific').dt.tz_convert('UTC')
devicelog['enddatetime'] = pd.to_datetime(devicelog['enddatetime']).dt.tz_localize('US/Pacific').dt.tz_convert('UTC')
devicelog['duration'] = pd.to_timedelta(devicelog['duration'], unit='second')
# create a col for real signal preemption
devicelog['event_start'] = devicelog['enddatetime'] - devicelog['duration']



In [14]:
start = '2020-11-11'
end = '2020-11-12'

In [15]:
# take only with TSP request
tripdatas = tripdatas[tripdatas['event'] == 'TSP request']
# join triplogs with tripdatas on 'uid'
logdatas = tripdatas.merge(triplogs, on='uid', how='inner')

# take only 1 day
mask = (logdatas['time'] > start) & (logdatas['time'] <= end)
logdatas = logdatas[mask]

mask = (devicelog['event_start'] > start) & (devicelog['event_start'] <= end)
devicelog = devicelog[mask]

In [16]:
# number of trips, which sent TSP requests
logdatas.uid.nunique()

505

In [17]:
# merge opticomdevicelog and joint dataset of triplogs and tripdatas
devicelogdatas = logdatas.merge(devicelog, on='vehiclename', how='inner')

In [18]:
# take only trips with 15 minutes interval between sending and preemption (and not less than zero!)
devicelogdatas['timedelta'] = devicelogdatas.event_start - devicelogdatas.time
mask = (devicelogdatas.timedelta < pd.Timedelta("15 minutes")) & \
        (devicelogdatas.timedelta >= pd.Timedelta("0 minutes"))
filtered_df = devicelogdatas[mask]


In [19]:
# number of trips after "advanced filtering"
filtered_df.uid.nunique()

465

In [20]:
# number of trips without any cleaning steps
mask = (triplogs['starttime'] > start) & (triplogs['starttime'] <= end)
triplogs[mask].uid.nunique()

1854